# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
%matplotlib inline

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
import gmaps
import gmaps.datasets
import os
import matplotlib.pyplot as plt
import json
import warnings
warnings.filterwarnings("ignore")
# Import API key
from apikeys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
# Display sample data
city_data_df.head()

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ribeira grande,38.5167,-28.7000,289.14,77,12,3.13,PT,1686446244
1,1,adamstown,-25.0660,-130.1015,293.71,55,53,5.73,PN,1686446244
2,2,nizhneudinsk,54.9076,99.0276,286.24,96,100,2.93,RU,1686446244
3,3,papatowai,-46.5619,169.4708,283.15,77,100,1.81,NZ,1686446244
4,4,puerto ayora,-0.7393,-90.3518,299.57,91,92,4.47,EC,1686446244


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display


map_plot_1=city_data_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size= "Humidity",
    color="City",
    alpha=.70)

# Display the map
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Drop any rows with null values
no_nons = city_data_df.dropna()

# Narrow down cities that fit criteria and drop any results with null values
zero_cloud = no_nons[no_nons["Cloudiness"] == 0]
humidity_low30 = zero_cloud[zero_cloud["Humidity"]<30]
windspeed_45 = humidity_low30[humidity_low30["Wind Speed"]<4.5]


# Display sample data
perfect_vacation = humidity_low30.reset_index()
del perfect_vacation["index"]
perfect_vacation

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,78,ad dilam,23.9915,47.1627,303.38,14,0,2.09,SA,1686446253
1,92,cabo san lucas,22.8909,-109.9124,306.82,29,0,5.14,MX,1686445989
2,104,kushk,33.2957,61.9522,295.06,29,0,7.73,AF,1686446256
3,119,i-n-salah,27.1935,2.4607,306.71,10,0,4.42,DZ,1686446258
4,165,mhamid,29.8200,-5.7200,300.45,22,0,5.26,MA,1686446264
5,184,mount isa,-20.7333,139.5000,298.02,25,0,4.12,AU,1686446266
6,205,shitanjing,39.2342,106.3439,295.60,19,0,3.97,CN,1686446269
7,207,bilibino,68.0546,166.4372,293.45,20,0,5.19,RU,1686446269
8,268,maun,-19.9833,23.4167,289.13,29,0,1.54,BW,1686446276
9,290,big spring,32.2504,-101.4787,307.65,14,0,3.13,US,1686446069


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API 

hotel_df = perfect_vacation
hotel_df["Hotel Name"] = " "
  
hotel_df  

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,78,ad dilam,23.9915,47.1627,303.38,14,0,2.09,SA,1686446253,
1,92,cabo san lucas,22.8909,-109.9124,306.82,29,0,5.14,MX,1686445989,
2,104,kushk,33.2957,61.9522,295.06,29,0,7.73,AF,1686446256,
3,119,i-n-salah,27.1935,2.4607,306.71,10,0,4.42,DZ,1686446258,
4,165,mhamid,29.8200,-5.7200,300.45,22,0,5.26,MA,1686446264,
5,184,mount isa,-20.7333,139.5000,298.02,25,0,4.12,AU,1686446266,
6,205,shitanjing,39.2342,106.3439,295.60,19,0,3.97,CN,1686446269,
7,207,bilibino,68.0546,166.4372,293.45,20,0,5.19,RU,1686446269,
8,268,maun,-19.9833,23.4167,289.13,29,0,1.54,BW,1686446276,
9,290,big spring,32.2504,-101.4787,307.65,14,0,3.13,US,1686446069,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000

categories= "accommodation.hotel"
params = {
   "categories":categories,
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng=hotel_df.loc[index, "Longitude"]
    lat= hotel_df.loc[index, "Latitude"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


ad dilam - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
kushk - nearest hotel: No hotel found
i-n-salah - nearest hotel: No hotel found
mhamid - nearest hotel: No hotel found
mount isa - nearest hotel: No hotel found
shitanjing - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
maun - nearest hotel: No hotel found
big spring - nearest hotel: No hotel found
reggane - nearest hotel: No hotel found
tamanrasset - nearest hotel: No hotel found
bela - nearest hotel: No hotel found
al kharijah - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
creel - nearest hotel: No hotel found
cortez - nearest hotel: No hotel found
holtville - nearest hotel: No hotel found
bam - nearest hotel: No hotel found
taiyuan - nearest hotel: No hotel found


,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,78,ad dilam,23.9915,47.1627,303.38,14,0,2.09,SA,1686446253,No hotel found
1,92,cabo san lucas,22.8909,-109.9124,306.82,29,0,5.14,MX,1686445989,No hotel found
2,104,kushk,33.2957,61.9522,295.06,29,0,7.73,AF,1686446256,No hotel found
3,119,i-n-salah,27.1935,2.4607,306.71,10,0,4.42,DZ,1686446258,No hotel found
4,165,mhamid,29.8200,-5.7200,300.45,22,0,5.26,MA,1686446264,No hotel found
5,184,mount isa,-20.7333,139.5000,298.02,25,0,4.12,AU,1686446266,No hotel found
6,205,shitanjing,39.2342,106.3439,295.60,19,0,3.97,CN,1686446269,No hotel found
7,207,bilibino,68.0546,166.4372,293.45,20,0,5.19,RU,1686446269,No hotel found
8,268,maun,-19.9833,23.4167,289.13,29,0,1.54,BW,1686446276,No hotel found
9,290,big spring,32.2504,-101.4787,307.65,14,0,3.13,US,1686446069,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot_2=hotel_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    color="City",
    scale=0.5,
    size= "Humidity",
    hover_cols = ["Hotel Name", "Country"]
    )

# Display the map
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity,Hotel Name,Country)